#SkyFusion

### Preliminary Imports and Google Drive Mounting

In [ ]:
import os
import requests
import zipfile
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Loading labels in original AiTOD v2 format



In [ ]:
file_zip1 = '/content/drive/MyDrive/labels_Sky_old/imagesTest.zip'
final_directory = '/content/LabelsArrangement'
with zipfile.ZipFile(file_zip1, 'r') as zip_ref:
    zip_ref.extractall(final_directory)

In [ ]:
file_zip2 = '/content/drive/MyDrive/labels_Sky_old/imagesTrain.zip'
with zipfile.ZipFile(file_zip2, 'r') as zip_ref:
    zip_ref.extractall(final_directory)

In [ ]:
file_zip3 = '/content/drive/MyDrive/labels_Sky_old/imagesVal.zip'
with zipfile.ZipFile(file_zip3, 'r') as zip_ref:
    zip_ref.extractall(final_directory)

In [ ]:
file_zip4 = '/content/drive/MyDrive/labels_Sky_old/labelsTest.zip'
with zipfile.ZipFile(file_zip4, 'r') as zip_ref:
    zip_ref.extractall(final_directory)

In [ ]:
file_zip5 = '/content/drive/MyDrive/labels_Sky_old/labelsTrain.zip'
with zipfile.ZipFile(file_zip5, 'r') as zip_ref:
    zip_ref.extractall(final_directory)

In [ ]:
file_zip6 = '/content/drive/MyDrive/labels_Sky_old/labelsVal.zip'
with zipfile.ZipFile(file_zip6, 'r') as zip_ref:
    zip_ref.extractall(final_directory)

### Converting annotations from AiTOD v2 to YOLO format for SkyFusion-Yolo dataset

In [ ]:
def convert_to_yolo_format(annotation, image_width, image_height):
    # Extract class index
    class_index = int(annotation[0])
    # Extract bb coordinates
    points = annotation[1:]

  # Calculate extrema of the bb
    min_x = min(points[0::2])
    max_x = max(points[0::2])
    min_y = min(points[1::2])
    max_y = max(points[1::2])

  # Find center of the bb
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2

  # Calculate width and heigth of bb
    width = max_x - min_x
    height = max_y - min_y

  # Define YOLO format string
    yolo_annotation = f"{class_index} {center_x} {center_y} {width} {height}"

    return yolo_annotation

In [ ]:
folder_images_Train = '/content/LabelsArrangement/imagesTrain'
folder_labels_Train = '/content/LabelsArrangement/labelsTrain'
folder_images_Valid = '/content/LabelsArrangement/imagesVal'
folder_labels_Valid = '/content/LabelsArrangement/labelsVal'
folder_images_Test = '/content/LabelsArrangement/imagesTest'
folder_labels_Test = '/content/LabelsArrangement/labelsTest'

try:
  # List all files in  target folders
    filesTrainLab = os.listdir(folder_labels_Train)
    filesValidLab = os.listdir(folder_labels_Valid)
    filesTrainImm = os.listdir(folder_images_Train)
    filesValidImm = os.listdir(folder_images_Valid)
    filesTestImm = os.listdir(folder_images_Test)
    filesTestLab = os.listdir(folder_labels_Test)
  # Order files
    filesTrainLab.sort()
    filesValidLab.sort()
    filesTestLab.sort()
    filesTrainImm.sort()
    filesValidImm.sort()
    filesTestImm.sort()

  # Write new labels for train set
    if filesTrainLab and filesTrainImm:
      for filep, filep2 in zip(filesTrainLab, filesTrainImm):

          pfLab = os.path.join(folder_labels_Train, filep)
          pfImg = os.path.join(folder_images_Train, filep2)
          image = cv2.imread(str(pfImg))
          height, width, _ = image.shape
          # Read file and extract info
          with open(pfLab, 'r+') as file:
              content = ""
              for line in file:
                  data = list(map(float, line.split()))
                  if data[0] != 0:
                      # Convert Line to YOLO format
                      yolo_annotation = convert_to_yolo_format(data, width, height)
                      content += yolo_annotation + "\n"
                  else:
                    content += line
            # Write new labels
              file.seek(0)
              file.write(content)
              file.truncate()
    else:
        print("Empty Folder")

 # Write new labels for validation set
    if filesValidLab and filesValidImm:
        for filep, filep2 in zip(filesValidLab, filesValidImm):
            pfLab = os.path.join(folder_labels_Valid, filep)
            pfImg = os.path.join(folder_images_Valid, filep2)
            image = cv2.imread(str(pfImg))
            height, width, _ = image.shape
             # Read file and extract info
            with open(pfLab, 'r+') as file:
              content = ""
              for line in file:
                  data = list(map(float, line.split()))
                  if data[0] != 0:
                      # Convert Line to YOLO format
                      yolo_annotation = convert_to_yolo_format(data, width, height)
                      content += yolo_annotation + "\n"
                  else:
                    content += line
           # Write new labels
              file.seek(0)
              file.write(content)
              file.truncate()
    else:
        print("Empty Folder")

# Write new labels for test set
    if filesTestLab and filesTestImm:
        for filep, filep2 in zip(filesTestLab, filesTestImm):
            pfLab = os.path.join(folder_labels_Test, filep)
            pfImg = os.path.join(folder_images_Test, filep2)
            image = cv2.imread(str(pfImg))
            height, width, _ = image.shape
             # Read file and extract info
            with open(pfLab, 'r+') as file:
              content = ""
              for line in file:
                  data = list(map(float, line.split()))
                  if data[0] != 0:
                      # Convert line to YOLO format
                      yolo_annotation = convert_to_yolo_format(data, width, height)
                      content += yolo_annotation + "\n"
                  else:
                    content += line
              # Write new labels
              file.seek(0)
              file.write(content)
              file.truncate()
    else:
        print("Empty Folder")
except FileNotFoundError:
    print(f"Folder {filesTrainLab} does not exist.")

In [ ]:
import os

# Path to old labels folder
oldLabels_folder = '/content/LabelsArrangement/labelsVal'

# Path to new labels folder
nuLabels_folder = '/content/drive/My Drive/labelsValNew'

# Copy comand from old to new folder
copy = f'cp -r "{oldLabels_folder}" "{nuLabels_folder}"'

# Execute command
os.system(comando_copia)

0

In [ ]:
# Path to old labels folder
oldLabels_folder = '/content/LabelsArrangement/labelsTest'

# Path to new labels folder
nuLabels_folder = '/content/drive/My Drive/labelsTestNew'

# Copy comand from old to new folder
copy = f'cp -r "{oldLabels_folder}" "{nuLabels_folder}"'

# Execute command
os.system(comando_copia)

0

In [ ]:
# Path to old labels folder
oldLabels_folder = '/content/LabelsArrangement/labelsTrain'

# Path to new labels folder
nuLabels_folder = '/content/drive/My Drive/labelsTrainNew'

# Copy comand from old to new folder
copy = f'cp -r "{oldLabels_folder}" "{nuLabels_folder}"'

# Execute command
os.system(comando_copia)

0

# GreenDetection
This code snippet is made for converting labels froma the Palm Tree Dataset, which are stored in a csv file and in a different format.

The idea is: crop the image in a square shape and minimize loss of trees in the picture. Also convert all values with respect to new coordinates and normalize them to march YOLO format.

In [ ]:
import csv
import cv2
import os

# Resize
def resize_image(input_path, output_path, target, left, top, right, bottom):
    # Create output folder if not existing
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    filename = target
    # Check if file is a supported image format
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        # Read the image in BGR color space (default for cv2)
        img = cv2.imread(os.path.join(input_path, filename))

        # Extract desired region based on provided coordinates (left, top, right, bottom)
        cropped_img = img[bottom:top, left:right]

        # Calculate dimensions of the cropped image
        height, width = cropped_img.shape[:2]

        # Determine new size to ensure square output (using the maximum dimension)
        new_size = max(height, width)

        # Resize the cropped image to a square with the calculated size
        resized_img = cv2.resize(cropped_img, (new_size, new_size), interpolation=cv2.INTER_AREA)

        # Save the resized and cropped image
        cv2.imwrite(os.path.join(output_path, filename), resized_img)


# Dictionary for storing data, with filenames as keys
data_by_filename = {}
csv_path = '/Users/bojack/Desktop/Palm/train_labels.csv'

# Open CSV file
with open(csv_path, newline='') as csvfile:
    # Read CSV file
    reader = csv.reader(csvfile, delimiter=',')

    # Skip header
    next(reader, None)

    # Iterate over csv rowss
    for row in reader:
        # Extract Line content
        filename, width, height, class_, xmin, ymin, xmax, ymax = row

        # Assign class integer
        if class_ == 'Tree':
            class_ = 0
        elif class_ == 'Palm':
            class_ = 1
        else:
            class_ = 2

        # Check if filename is already in dictionary
        if filename in data_by_filename:
            # Add info to data arrays
            data_by_filename[filename]['class'].append(class_)
            data_by_filename[filename]['xmins'].append(xmin)
            data_by_filename[filename]['ymins'].append(ymin)
            data_by_filename[filename]['xmaxs'].append(xmax)
            data_by_filename[filename]['ymaxs'].append(ymax)
        else:
            # Create filename entry in dictionary
            data_by_filename[filename] = {'class': [class_], 'width': width, 'height': height,'xmins': [xmin],'ymins': [ymin],'xmaxs': [xmax],'ymaxs': [ymax], 'content': ''}

# Modify info in dictionary to convert labels to YOLO format and to crop the image
    for filename, info in data_by_filename.items():
        classes = info['class']
        height = int(info['height'])
        width = int(info['width'])

        xmins = list(map(int, info['xmins']))
        xmaxs = list(map(int, info['xmaxs']))
        ymins = list(map(int, info['ymins']))
        ymaxs = list(map(int, info['ymaxs']))

      # Find first and last tree position
        min_tree = min(xmins)
        max_tree = max(xmaxs)

      # Set top and bottom margin
        top = height
        bottom = 0

        if min_tree + height > width:
          # Set cropping area to far right
            right = width
            left = width-height

        elif max_tree - height < 0:
           # Set cropping area to far left
            right = height
            left = 0


        else:
          # Find optimal cropping area (left-right position)
            max_count = 0
            xmin_max_count = -1

            for x in xmins:
                count = 0
                for element in xmaxs:
                    if x <= element <= x+height:
                        count += 1
                if count > max_count:
                    max_count = count
                    xmin_max_count = x

            if xmin_max_count + height > width:
                left = width-height
                right = width

            else:
                left = xmin_max_count
                right = xmin_max_count + height

        i = 0
      # Iterate to modify labels
        while i < len(info['xmins']):
          # Delete all labels out of cropping area
            if (int(info['xmins'][i]) < left) or (int(info['xmaxs'][i]) > right):
                del info['xmins'][i]
                del info['xmaxs'][i]
                del info['ymaxs'][i]
                del info['ymins'][i]
                del info['class'][i]
            else:
              # Recenter coordinates of bounding boxes
                min_x = int(info['xmins'][i]) - left
                max_x = int(info['xmaxs'][i]) - left
                min_y = int(info['ymins'][i])
                max_y = int(info['ymaxs'][i])

              # Finde center of bb
                center_x = (min_x + max_x) / 2 / height
                center_y = (min_y + max_y) / 2 / height
                class_index = info['class'][i]

              # Calculate height and width of bb
                widthb = (max_x - min_x)/ height
                heightb = (max_y - min_y)/ height

              # Convert to YOLO format
                yolo_annotation = f"{class_index} {center_x} {center_y} {widthb} {heightb}"
                data_by_filename[filename]['content'] = data_by_filename[filename]['content'] + yolo_annotation + "\n"
                i+=1

       # Save cropping coordinates
        data_by_filename[filename]['left'] = left
        data_by_filename[filename]['right'] = right
        data_by_filename[filename]['top'] = top
        data_by_filename[filename]['bottom'] = bottom

# Make new Labels folder if not existing
if not os.path.exists("/Users/bojack/Desktop/Palm/newLabels"):
    os.makedirs("/Users/bojack/Desktop/Palm/newLabels")

# Make new Pictures folder if not existing
if not os.path.exists("/Users/bojack/Desktop/Palm/newpics_train"):
    os.makedirs("/Users/bojack/Desktop/Palm/newpics_train")

# Iterate over dictionary again to create labels and crop images
for filename, info in data_by_filename.items():
    # Get cropping area coordinates
    right = info['right']
    left = info['left']
    top = info['top']
    bottom = info['bottom']

    input_path = "/Users/bojack/Desktop/Palm/train"
    output_folder = "/Users/bojack/Desktop/Palm/newpics_train"
    target = filename
    # Crop Images
    if os.path.exists(input_path+'/'+target):
        resize_image(input_path, output_folder, target, left, top, right, bottom)
        content = info['content']

      # Create .txt file for labels
        p = filename.rsplit(".", 1)[0]
        txt = "/Users/bojack/Desktop/Palm/newLabels/"+p+".txt"
        with open(txt, 'w') as txtfile:
        # Write the content in the text file
            txtfile.seek(0)
            txtfile.write(content)
            txtfile.truncate()
